In [9]:
import pandas as pd
df1 = pd.read_csv(r'C:\Users\Admin\Desktop\Codes\ISRO\data_2016.csv')

In [10]:
df1.head()

,Unnamed: 0,valid_time,latitude,longitude,u10,v10,d2m,t2m,number,expver,tp
0,0,2016-01-01 12:00:00,31.19,77.35,0.766122,0.334703,270.57650,284.43530,0,1,0.000000e+00
1,1,2016-01-01 12:00:00,31.19,77.60,0.217600,0.376696,267.45640,283.61108,0,1,0.000000e+00
2,2,2016-01-01 12:00:00,31.19,77.85,-0.547171,0.080431,263.43490,280.02612,0,1,0.000000e+00
3,3,2016-01-01 12:00:00,31.19,78.10,-0.858511,-0.191359,260.24155,275.30835,0,1,2.280576e-07
4,4,2016-01-01 12:00:00,31.19,78.35,-0.681448,-0.101698,255.02866,268.68140,0,1,1.659384e-06


In [11]:
df1.shape

(65880, 11)

In [12]:
df = df1.drop(columns=['Unnamed: 0','number','expver'])

In [13]:
df.head()

,valid_time,latitude,longitude,u10,v10,d2m,t2m,tp
0,2016-01-01 12:00:00,31.19,77.35,0.766122,0.334703,270.57650,284.43530,0.000000e+00
1,2016-01-01 12:00:00,31.19,77.60,0.217600,0.376696,267.45640,283.61108,0.000000e+00
2,2016-01-01 12:00:00,31.19,77.85,-0.547171,0.080431,263.43490,280.02612,0.000000e+00
3,2016-01-01 12:00:00,31.19,78.10,-0.858511,-0.191359,260.24155,275.30835,2.280576e-07
4,2016-01-01 12:00:00,31.19,78.35,-0.681448,-0.101698,255.02866,268.68140,1.659384e-06


In [14]:
import numpy as np

df['temp_C'] = df['t2m'] - 273.15
df['dew_C'] = df['d2m'] - 273.15

e_t = 6.112 * np.exp((17.67 * df['temp_C']) / (df['temp_C'] + 243.5))
e_d = 6.112 * np.exp((17.67 * df['dew_C']) / (df['dew_C'] + 243.5))

df['humidity'] = 100 * (e_d / e_t)

df['temp'] = df['t2m'] - 273.15

df['wind_speed'] = np.sqrt(df['u10']**2 + df['v10']**2)

df['rainfall'] = df['tp'] * 1000   

In [16]:
df['valid_time'] = pd.to_datetime(df['valid_time'])

df['day'] = df['valid_time'].dt.day
df['month'] = df['valid_time'].dt.month

In [18]:
X = df[['month','day','temp', 'humidity', 'wind_speed', 'rainfall']]

In [19]:
import math
import pandas as pd


class FWICLASS:
    def __init__(self, temp, rhum, wind, prcp):
        self.t = temp   # Temperature (°C)
        self.h = rhum   # Relative humidity (%)
        self.w = wind   # Wind speed (km/h)
        self.p = prcp   # Precipitation (mm)

    # ------------------ FFMC ------------------
    def FFMCcalc(self, ffmc0):
        mo = (147.2 * (101.0 - ffmc0)) / (59.5 + ffmc0)

        if self.p > 0.5:
            rf = self.p - 0.5
            if mo > 150.0:
                mo = (mo + 42.5 * rf * math.exp(-100.0 / (251.0 - mo)) *
                      (1.0 - math.exp(-6.93 / rf))) + \
                     (0.0015 * (mo - 150.0) ** 2) * math.sqrt(rf)
            else:
                mo = mo + 42.5 * rf * math.exp(-100.0 / (251.0 - mo)) * \
                     (1.0 - math.exp(-6.93 / rf))
            if mo > 250.0:
                mo = 250.0

        ed = 0.942 * (self.h ** 0.679) + \
             (11.0 * math.exp((self.h - 100.0) / 10.0)) + \
             0.18 * (21.1 - self.t) * (1.0 - 1.0 / math.exp(0.1150 * self.h))

        if mo < ed:
            ew = 0.618 * (self.h ** 0.753) + \
                 (10.0 * math.exp((self.h - 100.0) / 10.0)) + \
                 0.18 * (21.1 - self.t) * (1.0 - 1.0 / math.exp(0.115 * self.h))
            if mo <= ew:
                kl = 0.424 * (1.0 - ((100.0 - self.h) / 100.0) ** 1.7) + \
                     (0.0694 * math.sqrt(self.w)) * (1.0 - ((100.0 - self.h) / 100.0) ** 8)
                kw = kl * (0.581 * math.exp(0.0365 * self.t))
                m = ew - (ew - mo) / 10.0 ** kw
            else:
                m = mo
        elif mo == ed:
            m = mo
        else:  # mo > ed
            kl = 0.424 * (1.0 - (self.h / 100.0) ** 1.7) + \
                 (0.0694 * math.sqrt(self.w)) * (1.0 - (self.h / 100.0) ** 8)
            kw = kl * (0.581 * math.exp(0.0365 * self.t))
            m = ed + (mo - ed) / 10.0 ** kw

        ffmc = (59.5 * (250.0 - m)) / (147.2 + m)
        ffmc = min(max(ffmc, 0.0), 101.0)
        return ffmc

    def DMCcalc(self, dmc0, mth):
        el = [6.5, 7.5, 9.0, 12.8, 13.9, 13.9,
              12.4, 10.9, 9.4, 8.0, 7.0, 6.0]
        t = max(self.t, -1.1)
        rk = 1.894 * (t + 1.1) * (100.0 - self.h) * (el[mth - 1] * 0.0001)

        if self.p > 1.5:
            ra = self.p
            rw = 0.92 * ra - 1.27
            wmi = 20.0 + 280.0 / math.exp(0.023 * dmc0)
            if dmc0 <= 33.0:
                b = 100.0 / (0.5 + 0.3 * dmc0)
            elif dmc0 <= 65.0:
                b = 14.0 - 1.3 * math.log(dmc0)
            else:
                b = 6.2 * math.log(dmc0) - 17.2
            wmr = wmi + (1000 * rw) / (48.77 + b * rw)
            pr = 43.43 * (5.6348 - math.log(wmr - 20.0))
        else:
            pr = dmc0

        dmc = max(pr + rk, 1.0)
        return dmc

    def DCcalc(self, dc0, mth):
        fl = [-1.6, -1.6, -1.6, 0.9, 3.8, 5.8,
              6.4, 5.0, 2.4, 0.4, -1.6, -1.6]
        t = max(self.t, -2.8)
        pe = max((0.36 * (t + 2.8) + fl[mth - 1]) / 2, 0.0)

        if self.p > 2.8:
            ra = self.p
            rw = 0.83 * ra - 1.27
            smi = 800.0 * math.exp(-dc0 / 400.0)
            dr = dc0 - 400.0 * math.log(1.0 + ((3.937 * rw) / smi))
            dc = dr + pe if dr > 0 else dc0 + pe
        else:
            dc = dc0 + pe
        return dc

    def ISIcalc(self, ffmc):
        mo = 147.2 * (101.0 - ffmc) / (59.5 + ffmc)
        ff = 19.115 * math.exp(mo * -0.1386) * (1.0 + (mo ** 5.31) / 49300000.0)
        isi = ff * math.exp(0.05039 * self.w)
        return isi

    def BUIcalc(self, dmc, dc):
        if dmc <= 0.4 * dc:
            bui = (0.8 * dc * dmc) / (dmc + 0.4 * dc)
        else:
            bui = dmc - (1.0 - 0.8 * dc / (dmc + 0.4 * dc)) * \
                  (0.92 + (0.0114 * dmc) ** 1.7)
        return max(bui, 0.0)

    def FWIcalc(self, isi, bui):
        if bui <= 80.0:
            bb = 0.1 * isi * (0.626 * bui ** 0.809 + 2.0)
        else:
            bb = 0.1 * isi * (1000.0 / (25.0 + 108.64 / math.exp(0.023 * bui)))
        if bb <= 1.0:
            fwi = bb
        else:
            fwi = math.exp(2.72 * (0.434 * math.log(bb)) ** 0.647)
        return fwi


def calculate_fwi_df(df):
    """
    Takes a DataFrame with columns ['month','day','temp','humidity','wind_speed','rainfall']
    and adds columns ['FFMC','DMC','DC','ISI','BUI','FWI'].
    """
    ffmc0, dmc0, dc0 = 85.0, 6.0, 15.0

    FFMC_list, DMC_list, DC_list, ISI_list, BUI_list, FWI_list = [], [], [], [], [], []

    for idx, row in df.iterrows():
        temp = row['temp']
        rhum = min(row['humidity'], 100.0)
        wind = row['wind_speed']
        prcp = row['rainfall']
        mth = int(row['month'])

        fwisystem = FWICLASS(temp, rhum, wind, prcp)

        ffmc = fwisystem.FFMCcalc(ffmc0)
        dmc = fwisystem.DMCcalc(dmc0, mth)
        dc = fwisystem.DCcalc(dc0, mth)
        isi = fwisystem.ISIcalc(ffmc)
        bui = fwisystem.BUIcalc(dmc, dc)
        fwi = fwisystem.FWIcalc(isi, bui)

        ffmc0, dmc0, dc0 = ffmc, dmc, dc

        FFMC_list.append(ffmc)
        DMC_list.append(dmc)
        DC_list.append(dc)
        ISI_list.append(isi)
        BUI_list.append(bui)
        FWI_list.append(fwi)

    df['FFMC'] = FFMC_list
    df['DMC'] = DMC_list
    df['DC'] = DC_list
    df['ISI'] = ISI_list
    df['BUI'] = BUI_list
    df['FWI'] = FWI_list

    return df

In [20]:
X = df[['month','day','temp','humidity','wind_speed','rainfall']]

X = calculate_fwi_df(X)

print(X)



       month  day      temp   humidity  wind_speed  rainfall       FFMC  \
0          1    1  11.28530  37.854922    0.836044  0.000000  86.763020   
1          1    1  10.46108  31.634725    0.435028  0.000000  88.133046   
2          1    1   6.87612  29.535902    0.553051  0.000000  88.640722   
3          1    1   2.15835  31.841175    0.879579  0.000228  88.689631   
4          1    1  -4.46860  33.608056    0.688995  0.001659  88.492688   
...      ...  ...       ...        ...         ...       ...        ...   
65875     12   31  17.14956  80.025576    2.553737  0.000000  82.903647   
65876     12   31  17.20474  80.888365    2.336540  0.000000  82.726570   
65877     12   31  17.80728  77.789077    2.003419  0.000000  82.773526   
65878     12   31  18.54750  74.944204    1.664949  0.000000  82.820498   
65879     12   31  19.22573  71.915920    1.375130  0.000000  83.138453   

                DMC            DC       ISI           BUI        FWI  
0          6.947560     16.7

C:\Users\Admin\AppData\Local\Temp\ipykernel_3888\1582297104.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FFMC'] = FFMC_list
C:\Users\Admin\AppData\Local\Temp\ipykernel_3888\1582297104.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DMC'] = DMC_list
C:\Users\Admin\AppData\Local\Temp\ipykernel_3888\1582297104.py:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [21]:
X.head()

,month,day,temp,humidity,wind_speed,rainfall,FFMC,DMC,DC,ISI,BUI,FWI
0,1,1,11.28530,37.854922,0.836044,0.000000,86.763020,6.947560,16.735354,2.813111,6.930220,2.202486
1,1,1,10.46108,31.634725,0.435028,0.000000,88.133046,7.920592,18.322348,3.352461,7.884247,3.046742
2,1,1,6.87612,29.535902,0.553051,0.000000,88.640722,8.612507,19.264050,3.627260,8.560303,3.522762
3,1,1,2.15835,31.841175,0.879579,0.000228,88.689631,8.885917,19.356553,3.713415,8.821260,3.684949
4,1,1,-4.46860,33.608056,0.688995,0.001659,88.492688,8.885917,19.356553,3.575415,8.821260,3.527760


In [25]:
X.shape

(65880, 12)

In [26]:
df.shape

(65880, 16)

In [27]:
df['FWI'] = X['FWI']

In [28]:
df.to_csv('data_2016_FWI.csv')